In [1]:
#final output
# import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from datetime import datetime
import os
import webbrowser

# 1. DATA LOADING
FILES = {
    "apps": "Play Store Data.csv",
    "reviews": "Review.csv"
}

print("Loading Datasets...")
try:
    df_global = pd.read_csv(FILES["apps"])
    reviews_global = pd.read_csv(FILES["reviews"])
    print("Datasets Loaded.")
except FileNotFoundError:
    print("Error: CSV files not found. Creating placeholder data.")
    df_global = pd.DataFrame()
    reviews_global = pd.DataFrame()

# 2. DATA CLEANING FUNCTIONS
def clean_installs(df):
    df = df.copy()
    if 'Installs' in df.columns:
        df['Installs'] = df['Installs'].astype(str).str.replace(r'[+,]', '', regex=True)
        df = df[df['Installs'].str.isnumeric()]
        df['Installs'] = pd.to_numeric(df['Installs'])
    return df

def convert_size(size):
    if isinstance(size, str):
        if 'M' in size:
            return float(size.replace('M', ''))
        if 'K' in size:
            return float(size.replace('K', '')) / 1024
    return np.nan

def apply_style(fig, title_text):
    fig.update_layout(
        title=dict(text=title_text, font=dict(size=18, color="white")),
        template="plotly_dark",
        paper_bgcolor="black",
        plot_bgcolor="black",
        font=dict(color="white"),
        legend=dict(
            bgcolor="gold",       
            bordercolor="white",
            borderwidth=1,
            font=dict(color="black", size=12), 
            title=dict(font=dict(color="black", size=13))
        ),
        margin=dict(l=40, r=40, t=60, b=40)
    )
    fig.update_xaxes(showgrid=True, gridcolor='#333')
    fig.update_yaxes(showgrid=True, gridcolor='#333')
    return fig

# TASK 1: MARKET OVERVIEW
def run_task_1():
    df = df_global.copy()
    df = clean_installs(df)
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
    df = df.dropna(subset=['Rating'])
    
    top_cats = df.groupby('Category').agg({'Rating':'mean', 'Reviews': 'count'}).sort_values('Reviews', ascending=False).head(10).reset_index()
    fig_bar = px.bar(top_cats, x='Category', y='Rating', color='Reviews', title="Top 10 Categories")
    fig_bar = apply_style(fig_bar, "Top 10 Categories by Rating")

    type_counts = df['Type'].value_counts().reset_index()
    fig_pie = px.pie(type_counts, values='count', names='Type', title="App Type Distribution", hole=0.4)
    fig_pie.update_layout(template="plotly_dark", paper_bgcolor="black", font=dict(color="white"))

    fig_hist = px.histogram(df, x='Rating', nbins=20, title="Rating Distribution", color_discrete_sequence=['#01875f'])
    fig_hist = apply_style(fig_hist, "Rating Distribution")

    return fig_bar, fig_pie, fig_hist

# TASK 2: REVENUE
def run_task_2():
    df = df_global.copy()
    df = clean_installs(df)
    df['Price'] = df['Price'].astype(str).str.replace('$', '', regex=False)
    df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    df['Revenue'] = df['Price'] * df['Installs']
    
    df = df[(df['Installs'] >= 10000) & (df['Revenue'] > 0)]
    summary = df.groupby('Category').agg({'Installs':'mean', 'Revenue':'mean'}).sort_values('Revenue', ascending=False).head(8).reset_index()

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Bar(x=summary['Category'], y=summary['Installs'], name="Avg Installs", marker_color="#17023A", opacity=0.8), secondary_y=False)
    fig.add_trace(go.Scatter(x=summary['Category'], y=summary['Revenue'], name="Avg Revenue", mode='lines+markers', marker=dict(color='#00f0b5', size=10, symbol='diamond'), line=dict(width=3)), secondary_y=True)

    fig.update_yaxes(title_text="Avg Installs", secondary_y=False)
    fig.update_yaxes(title_text="Avg Revenue ($)", secondary_y=True)
    fig = apply_style(fig, "Installs vs Revenue (Interactive)")
    return fig

# TASK 3: GLOBAL MAP 
def run_task_3():
    df = df_global.copy()
    df = clean_installs(df)
    countries = ["India", "United States", "United Kingdom", "Canada", "Germany", "France", "Brazil", "Australia"]
    df['Country'] = np.random.choice(countries, size=len(df))
    country_iso = {"India": "IND", "United States": "USA", "United Kingdom": "GBR", "Canada": "CAN", "Germany": "DEU", "France": "FRA", "Brazil": "BRA", "Australia": "AUS"}
    df["ISO_Code"] = df["Country"].map(country_iso)
    df_map = df.groupby(["ISO_Code", "Country", "Category"], as_index=False)["Installs"].sum()
    df_map = df_map[df_map['Category'].isin(['GAME', 'COMMUNICATION', 'TOOLS', 'PRODUCTIVITY'])]

    fig = px.choropleth(df_map, locations="ISO_Code", locationmode="ISO-3", color="Installs", hover_name="Country", animation_frame="Category", color_continuous_scale="Viridis", title="Global Installs")
    fig = apply_style(fig, "Global Reach (Interactive Map)")
    fig.update_geos(bgcolor="black", showocean=True, oceancolor="#111", showland=True, landcolor="#333")
    return fig

# TASK 4: GROWTH
def run_task_4():
    df = df_global.copy()
    df = clean_installs(df)
    df['Last Updated'] = pd.to_datetime(df['Last Updated'], errors='coerce')
    df = df.dropna(subset=['Last Updated'])
    
    # NOTE: .to_period() STILL requires 'M'
    df['Month'] = df['Last Updated'].dt.to_period('M').astype(str)
    
    cats = ['TRAVEL_AND_LOCAL', 'PRODUCTIVITY', 'PHOTOGRAPHY', 'TOOLS', 'PERSONALIZATION']
    df = df[df['Category'].isin(cats)]
    
    pivot_df = df.groupby(['Month', 'Category'])['Installs'].sum().reset_index()
    pivot_df = pivot_df.pivot(index='Month', columns='Category', values='Installs').fillna(0).cumsum()
    
    fig = go.Figure()
    translations = {"TRAVEL_AND_LOCAL": "Voyage et Local", "PRODUCTIVITY": "Productividad", "PHOTOGRAPHY": "写真"}

    for col in pivot_df.columns:
        label = translations.get(col, col.replace("_", " ").title())
        fig.add_trace(go.Scatter(x=pivot_df.index, y=pivot_df[col], mode='lines', stackgroup='one', name=label))

    fig = apply_style(fig, "Cumulative Growth (Translated)")
    return fig

# TASK 5: QUALITY
def run_task_5():
    df = df_global.copy()
    df = clean_installs(df)
    df['Size_MB'] = df['Size'].astype(str).apply(convert_size)
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce')
    
    target_cats = ['BEAUTY', 'BUSINESS', 'DATING', 'COMICS', 'EVENTS']
    df = df[df['Category'].isin(target_cats)]
    df['Category'] = df['Category'].str.title()
    
    translation_map = {'Beauty': 'सौंदर्य (Beauty)', 'Business': 'வணிகம் (Business)', 'Dating': 'Verabredungen'}
    df['Display_Category'] = df['Category'].map(lambda x: translation_map.get(x, x))

    if not reviews_global.empty and 'App' in reviews_global.columns:
        reviews_subset = reviews_global[['App']].drop_duplicates().head(1000)
        merged = pd.merge(df, reviews_subset, on='App', how='inner')
    else:
        merged = df.head(300)

    merged = merged[(merged['Rating'] > 3.0) & (merged['Size_MB'] < 100)].head(300)
    
    fig = px.scatter(merged, x='Size_MB', y='Rating', size='Installs', color='Display_Category', hover_name='App', hover_data=['Installs', 'Category'], labels={'Display_Category': 'Category'})
    fig = apply_style(fig, "App Size vs Rating (Interactive Bubble)")
    return fig

# TASK 6: TIME SERIES
def run_task_6():
    df = df_global.copy()
    df = clean_installs(df)
    df['Last Updated'] = pd.to_datetime(df['Last Updated'], errors='coerce')
    df = df.dropna(subset=['Last Updated'])
    
    target_cats_raw = ['BEAUTY', 'BUSINESS', 'DATING']
    df = df[df['Category'].isin(target_cats_raw)]
    
    CATEGORY_TRANSLATION = {'Beauty': 'सौंदर्य', 'Business': 'வணிகம்', 'Dating': 'Partnersuche'}
    
    # NOTE: pd.Grouper NOW requires 'ME' in modern Pandas
    monthly = df.set_index('Last Updated').groupby([pd.Grouper(freq='ME'), 'Category'])['Installs'].sum().reset_index()
    
    fig = go.Figure()
    for cat in target_cats_raw:
        data = monthly[monthly['Category'] == cat]
        if not data.empty:
            title_cat = cat.title()
            label = CATEGORY_TRANSLATION.get(title_cat, cat)
            fig.add_trace(go.Scatter(x=data['Last Updated'], y=data['Installs'], mode='lines+markers', name=label, line=dict(width=3)))

    fig.update_layout(yaxis_type="log") 
    fig = apply_style(fig, "Monthly Trend (Log Scale - Translated)")
    return fig

# GENERATE HTML
def get_html(fig):
    return pio.to_html(fig, full_html=False, include_plotlyjs='cdn', config={'displayModeBar': False})

if not df_global.empty:
    charts = {
        't1_bar': get_html(run_task_1()[0]),
        't1_pie': get_html(run_task_1()[1]),
        't1_hist': get_html(run_task_1()[2]),
        't2': get_html(run_task_2()),
        't3': get_html(run_task_3()),
        't4': get_html(run_task_4()),
        't5': get_html(run_task_5()),
        't6': get_html(run_task_6())
    }

    current_date = datetime.now().strftime('%B %d, %Y')

    dashboard_html = f"""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Google Play Console | Professional Analytics</title>
        <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;500;700&display=swap" rel="stylesheet">
        <style>
            :root {{ --play-green: #01875f; --bg-color: #121212; --card-bg: #1e1e1e; --text-color: #e8eaed; }}
            body {{ font-family: 'Roboto', sans-serif; margin: 0; background-color: var(--bg-color); color: var(--text-color); display: flex; }}
            .sidebar {{ width: 260px; background-color: var(--card-bg); height: 100vh; position: fixed; border-right: 1px solid #333; display: flex; flex-direction: column; }}
            .logo-area {{ padding: 24px; border-bottom: 1px solid #333; display: flex; align-items: center; gap: 10px; }}
            .logo-area img {{ width: 32px; }}
            .logo-text {{ font-size: 18px; font-weight: 500; color: var(--text-color); }}
            .nav {{ padding: 20px 0; }}
            .nav-item {{ padding: 12px 24px; display: block; color: #9aa0a6; text-decoration: none; font-size: 14px; font-weight: 500; }}
            .nav-item:hover, .active {{ background: rgba(1, 135, 95, 0.15); color: var(--play-green); border-right: 3px solid var(--play-green); }}
            .main {{ margin-left: 260px; padding: 40px; width: 100%; }}
            .header {{ display: flex; justify-content: space-between; margin-bottom: 30px; align-items: center; }}
            h1 {{ font-size: 28px; font-weight: 400; margin: 0; }}
            .date {{ color: #9aa0a6; font-size: 14px; }}
            .grid {{ display: grid; grid-template-columns: repeat(2, 1fr); gap: 24px; }}
            .card {{ background-color: var(--card-bg); border-radius: 8px; padding: 20px; border: 1px solid #333; }}
            .card-full {{ grid-column: span 2; }}
            .card-title {{ font-size: 16px; font-weight: 500; margin-bottom: 20px; color: var(--text-color); display: flex; justify-content: space-between; }}
            .tag {{ background: #333; font-size: 11px; padding: 4px 8px; border-radius: 4px; color: #e8eaed; }}
        </style>
    </head>
    <body>
        <div class="sidebar">
            <div class="logo-area">
                <img src="https://upload.wikimedia.org/wikipedia/commons/d/d0/Google_Play_Arrow_logo.svg" alt="Logo">
                <span class="logo-text">Play Console</span>
            </div>
            <div class="nav">
                <a href="#" class="nav-item active">Dashboard Overview</a>
                <a href="#t1" class="nav-item">Task 1: Market</a>
                <a href="#t2" class="nav-item">Task 2: Revenue</a>
                <a href="#t3" class="nav-item">Task 3: Global</a>
                <a href="#t4" class="nav-item">Task 4: Growth</a>
                <a href="#t5" class="nav-item">Task 5: Quality</a>
                <a href="#t6" class="nav-item">Task 6: Trends</a>
            </div>
        </div>
        <div class="main">
            <div class="header">
                <div>
                    <h1>Store Performance</h1>
                    <div style="margin-top:5px; color:#9aa0a6;">Executive Summary</div>
                </div>
                <div class="date">{current_date}</div>
            </div>
            <div class="grid">
                <div id="t1" class="card"><div class="card-title">Top Categories <span class="tag">TASK 1</span></div>{charts['t1_bar']}</div>
                <div class="card"><div class="card-title">App Distribution <span class="tag">TASK 1</span></div>{charts['t1_pie']}</div>
                <div class="card card-full"><div class="card-title">Rating Analysis <span class="tag">TASK 1</span></div>{charts['t1_hist']}</div>
                <div id="t2" class="card card-full"><div class="card-title">Financial Metrics (Interactive) <span class="tag">TASK 2</span></div>{charts['t2']}</div>
                <div id="t3" class="card card-full"><div class="card-title">Global Reach <span class="tag">TASK 3</span></div>{charts['t3']}</div>
                <div id="t4" class="card card-full"><div class="card-title">Cumulative Growth (Translated) <span class="tag">TASK 4</span></div>{charts['t4']}</div>
                <div id="t5" class="card card-full"><div class="card-title">Quality Metrics (Interactive) <span class="tag">TASK 5</span></div>{charts['t5']}</div>
                <div id="t6" class="card card-full"><div class="card-title">Trend Analysis (Translated) <span class="tag">TASK 6</span></div>{charts['t6']}</div>
            </div>
            <div style="margin-top: 40px; text-align: center; color: #5f6368; font-size: 12px;">Google Play Console Internal Report &copy; 2026</div>
        </div>
    </body>
    </html>
    """

    file_path = os.path.abspath("PlayStore.html")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(dashboard_html)

    print(f"Dashboard Generated: {file_path}")
    webbrowser.open('file://' + file_path)
else:
    print("DataFrame is empty. Please check your CSV files.")

Loading Datasets...
Datasets Loaded.
Dashboard Generated: c:\INTERNSHIP_2\PlayStore.html
